In [1]:
#Retirando os dados do Kaggle
import kagglehub
# Download latest version
path = kagglehub.dataset_download("olistbr/brazilian-ecommerce")

print("Path to dataset files:", path)

100%|██████████| 42.6M/42.6M [00:00<00:00, 145MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/olistbr/brazilian-ecommerce/versions/2


**Conjunto de dados públicos de comércio eletrônico brasileiro da Olist**

link: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce/data

### 1 - Carregando os dados e bibliotecas

In [2]:
#Importando as bibliotecas que irão ser usadas

In [3]:
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 4.8 MB/s eta 0:00:00


In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from unidecode import unidecode #Usado na decodificação

import warnings #Para silenciar os avisos
warnings.filterwarnings('ignore')

# Carregando o conjunto de dados

In [40]:
clientes = pd.read_csv(f'{path}/olist_customers_dataset.csv')
geolocation = pd.read_csv(f'{path}/olist_geolocation_dataset.csv')
pedidos_itens = pd.read_csv(f'{path}//olist_order_items_dataset.csv')
pedidos_pagamentos = pd.read_csv(f'{path}//olist_order_payments_dataset.csv')
pedidos_reviews = pd.read_csv(f'{path}//olist_order_reviews_dataset.csv')
pedidos = pd.read_csv(f'{path}//olist_orders_dataset.csv')
produtos = pd.read_csv(f'{path}//olist_products_dataset.csv')
vendedores = pd.read_csv(f'{path}//olist_sellers_dataset.csv')
#Caso queira traduzir para o ingês é so descomentar abaixo
#product_category_name_translation = pd.read_csv('//product_category_name_translation.csv')

## Organizando os dados

Cliente

In [41]:
#Removendo os espaços em branco e captalizando(Deixando a primeira letra maiúscula)
clientes['customer_city'] = clientes['customer_city'].str.strip().str.capitalize()
#Deixando as abreviações da coluna estado no UPPERCASE
clientes['customer_state'] = clientes['customer_state'].str.upper()
#Convertendo o CEP para String
#str.zfill(5) uma alterntiva caso queira deixar os 5 números do código posta
clientes['customer_zip_code_prefix'] = clientes['customer_zip_code_prefix'].astype(str)

In [42]:
clientes.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,Franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,Sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,Sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,Mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,Campinas,SP


Geolocalização

api para a geolocalização:
https://servicodados.ibge.gov.br/api/docs/localidades?versao=1

In [43]:
#Normalizando o dados da geolocalização
geolocation['geolocation_city'] = geolocation['geolocation_city'].apply(unidecode).str.lower().str.strip()


In [44]:
geolocation.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


Itens do pedido

In [45]:
#Transformando a coluna em um datetime
pedidos_itens['shipping_limit_date'] = pd.to_datetime(pedidos_itens['shipping_limit_date'])
#Calculado o preço total do produto(frete + valor do produto)
pedidos_itens['preco_total'] = pedidos_itens['price'] + pedidos_itens['freight_value']
#"Juntando" os valores do pedido com o id do pedido
#Essa coluna pode ser usada para análises análises separadas
pedido_preco = pedidos_itens.groupby('order_id')['preco_total'].sum().reset_index()

In [46]:
pedido_preco.head()

,order_id,preco_total
0,00010242fe8c5a6d1ba2dd792cb16214,72.19
1,00018f77f2f0320c557190d7a144bdd3,259.83
2,000229ec398224ef6ca0657da4fc703e,216.87
3,00024acbcdf0a6daa1e931b038114c75,25.78
4,00042b26cf59d7ce69dfabb4e55b4fd9,218.04


In [47]:
pedidos_itens.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,preco_total
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,72.19
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,259.83
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,216.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,25.78
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,218.04


Pagamentos de pedidos

In [59]:
#Sumarizando  pagamento por pedido
resumo_financeiro_pedidos = pedidos_pagamentos.groupby('order_id').agg({
    'payment_value' : 'sum',#Soma todos os valores parciais de pagamento
    'payment_sequential' : 'max', #Numero máximo de pagamentos sequencial(se foi em 3x o max será 3)
    'payment_installments' : 'max', # Identifica o maior número de parcelas
    'payment_type' : lambda x: list(x.unique()), #pega todos os pagamento usados em um lista única
}).reset_index()

In [60]:
resumo_financeiro_pedidos.head()

,order_id,payment_value,payment_sequential,payment_installments,payment_type
0,00010242fe8c5a6d1ba2dd792cb16214,72.19,1,2,[credit_card]
1,00018f77f2f0320c557190d7a144bdd3,259.83,1,3,[credit_card]
2,000229ec398224ef6ca0657da4fc703e,216.87,1,5,[credit_card]
3,00024acbcdf0a6daa1e931b038114c75,25.78,1,2,[credit_card]
4,00042b26cf59d7ce69dfabb4e55b4fd9,218.04,1,3,[credit_card]


Avaliações de Pedidos

In [50]:
#Convertendo os valores dos reviews para o date Time  e garantindo a formatção de datas desejadas
pedidos_reviews['review_creation_date'] = pd.to_datetime(pedidos_reviews['review_creation_date'], format="%Y-%m-%d %H:%M:%S")
pedidos_reviews['review_answer_timestamp'] = pd.to_datetime(pedidos_reviews['review_answer_timestamp'], format="%Y-%m-%d %H:%M:%S")


In [51]:
pedidos_reviews.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01,2018-03-02 10:26:53


Pedidos

In [52]:
#Convertendo todas as colunas(Com data) do pedido usando o "to_datetime()"
pedidos['order_purchase_timestamp']  = pd.to_datetime(pedidos['order_purchase_timestamp'])#momento exato em que o cliente clicou em "Comprar"
pedidos['order_approved_at']  = pd.to_datetime(pedidos['order_approved_at'])#Quando o pagamento foi confirmado pelo sistema
pedidos['order_delivered_carrier_date']  = pd.to_datetime(pedidos['order_delivered_carrier_date'])#Quando o pacote foi entregue
pedidos['order_delivered_customer_date']  = pd.to_datetime(pedidos['order_delivered_customer_date'])#Datareal em que o cliente recebeu o pedido em mão
pedidos['order_estimated_delivery_date']  = pd.to_datetime(pedidos['order_estimated_delivery_date'])#data de entrega que foi prometida ao cliente

# Calculando o tempo, do momento em que o cliente clicou em 'comprar' até o momento em que o pagamento foi confirmado
#Tempo de "demora(delay) em horas"
pedidos['approval_delay_hours'] = (
    pedidos['order_approved_at'] - pedidos['order_purchase_timestamp']
).dt.total_seconds() / 3600

In [53]:
pedidos.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,approval_delay_hours
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,0.178333
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,30.713889
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,0.276111
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,0.298056
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,1.030556


Produtos

In [54]:
#Calculando o volume de cada produto em cm cúbico
#Cálculo usado (V =COMPRIMENTO X ALTURA X LARGURA)
produtos ['product_volume_cm3'] = produtos ['product_length_cm'] *\
                                      produtos ['product_height_cm']*\
                                      produtos ['product_width_cm']

#Retirando as colunas de altura(height), largura(width) e comprimento(lenght)
produtos.drop (columns = ['product_length_cm' , 'product_height_cm' ,'product_width_cm' ]
                       , inplace = True ,errors = 'ignore')

In [55]:
produtos.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_volume_cm3
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,2240.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,10800.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,2430.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,2704.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,4420.0


Vendedores

In [56]:
#Fazendo a mesma limpeza feita em Clientes
vendedores['seller_city'] = vendedores['seller_city'].str.strip().str.capitalize()

#Acrescentando os 5 dígitos do cep
vendedores['seller_zip_code_prefix'] = vendedores['seller_zip_code_prefix'].astype(str).str.zfill(5)

In [57]:
vendedores.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,Campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,Mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,Rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,04195,Sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,Braganca paulista,SP


#Salvando os dados

### Opção 1
## **Dados separados**

Todas as tabelas que estão sendo ultilizadas
- clientes
- geolocation
- pedidos_itens
- pedidos_pagamentos
- pedidos_reviews
- pedidos
- produtos
- vendedores

In [ ]:
dados_a_salvar = {
    "clientes" : clientes,
    "geolocation" : geolocation,
    "pedidos_itens" : pedidos_itens,
    "pedidos_pagamentos" : pedidos_pagamentos,
    "pedidos_reviews" : pedidos_reviews,
    "pedidos": pedidos,
    "produtos" : produtos,
    "vendedores" : vendedores
}

In [ ]:
for nome_arquivo, df in dados_a_salvar.items():
  df.to_csv(f"{nome_arquivo}_processado.csv", index=False, encoding='utf-8')
  print(f"O arquivo {nome_arquivo} foi salvo com sucesso!")

O arquivo clientes foi salvo com sucesso!
O arquivo geolocation foi salvo com sucesso!
O arquivo pedidos_itens foi salvo com sucesso!
O arquivo pedidos_pagamentos foi salvo com sucesso!
O arquivo pedidos_reviews foi salvo com sucesso!
O arquivo pedidos foi salvo com sucesso!
O arquivo produtos foi salvo com sucesso!
O arquivo vendedores foi salvo com sucesso!


### Opção 2
## **Dados juntos**

**O Plano de União** (Esquema de IDs)
Para **criar** uma "*Tabela Mestra*" sem erros, seguindo a lógica das chaves (IDs):

- 1 - pedidos é o nosso ponto central.
- 2 - Unimos **pedidos_itens** aos **pedidos** via **order_id**.
- 3 - Unimos **clientes** aos **pedidos** via **customer_id**.
- 4 - Unimos **produtos** aos **pedidos_itens** via **product_id**.
- 5 - Unimos **vendedores** aos **pedidos_itens** via **seller_id**.

In [61]:
#1 Unindo o "pedido_itens" ao pedido
dados_final = pedidos.merge(pedidos_itens, on='order_id', how='left')

#2 acrescentando os pagamentos
dados_final = dados_final.merge(resumo_financeiro_pedidos, on='order_id', how='left')

#3 Acrescentando os reviews
dados_final = dados_final.merge(pedidos_reviews, on='order_id', how='left')

#4 acrescentando o clientes via Custumer_id
dados_final = dados_final.merge(clientes, on='customer_id', how='left')

#5 acrescentando o produto via produc_id
dados_final = dados_final.merge(produtos, on='product_id', how='left')

#6
dados_final = dados_final.merge(vendedores, on='seller_id', how='left')

In [62]:
dados_final.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'approval_delay_hours', 'order_item_id', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value', 'preco_total',
       'payment_value', 'payment_sequential', 'payment_installments',
       'payment_type', 'review_id', 'review_score', 'review_comment_title',
       'review_comment_message', 'review_creation_date',
       'review_answer_timestamp', 'customer_unique_id',
       'customer_zip_code_prefix', 'customer_city', 'customer_state',
       'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_volume_cm3', 'seller_zip_code_prefix', 'seller_city',
       'seller_state'],
      dtype='object')

In [63]:
dados_final.shape

(114092, 39)

In [64]:
dados_final.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,approval_delay_hours,order_item_id,...,customer_state,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_volume_cm3,seller_zip_code_prefix,seller_city,seller_state
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,0.178333,1.0,...,SP,utilidades_domesticas,40.0,268.0,4.0,500.0,1976.0,09350,Maua,SP
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,30.713889,1.0,...,BA,perfumaria,29.0,178.0,1.0,400.0,4693.0,31570,Belo horizonte,SP
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,0.276111,1.0,...,GO,automotivo,46.0,232.0,1.0,420.0,9576.0,14840,Guariba,SP
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,0.298056,1.0,...,RN,pet_shop,59.0,468.0,3.0,450.0,6000.0,31842,Belo horizonte,MG
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,1.030556,1.0,...,SP,papelaria,38.0,316.0,4.0,250.0,11475.0,08752,Mogi das cruzes,SP


In [65]:
#Salvando os dados
dados_final.to_csv("dados_totais.csv", index=False, encoding='utf-8')

#### Algumas referências que foram usadas

sudeste = ['SP', 'RJ', 'ES','MG']
nordeste= ['MA', 'PI', 'CE', 'RN', 'PE', 'PB', 'SE', 'AL', 'BA']
norte =  ['AM', 'RR', 'AP', 'PA', 'TO', 'RO', 'AC']
centro_oeste = ['MT', 'GO', 'MS' ,'DF' ]
sul = ['SC', 'RS', 'PR']

sql chalenge: https://www.kaggle.com/code/terencicp/sql-challenge-e-commerce-data-analysis

link: https://www.kaggle.com/code/gsdeepakkumar/e-commerce-dataset-analysis